[모델 Autoencoder와 softmax classifier를 결합]
1. Autoencoder를 사용해 데이터를 효율적으로 압축하고 특징을 추출한 후, 그 결과를 ANN으로 전달해 최종 분류를 수행
- 은닉층 3개로 구성
- 가우시안 분포(정규분포)에서 임의의 값 추출, sigmoid 활성화 함수 사용
- softmax 분류기
2. 손실함수 : MSE & cross-entropy
3. 옵티마이저
- RMSProp Optimizer (learning rate=0.01)
- Gradient Descent Optimizer (learning rate=0.1)
4. 학습 epoch=40
- batch size=32

In [61]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [62]:
# Data
def load_csv(file_path):

    # CSV 파일 로드
    data = np.loadtxt(file_path, delimiter=',', skiprows=1)
    
    # 입력 변수와 목표 변수 분리 ('fetal_health'를 목표 변수로 설정)
    x_data = data[:, :-1]  # 마지막 열 제외 (입력 변수)
    y_data = data[:, -1].astype(int)  # 마지막 열 (목표 변수)
    
    # 목표 변수를 one-hot 인코딩으로 변환
    with tf.Session() as sess:
        y_data = sess.run(tf.one_hot(y_data - 1, depth=3))

    return x_data, y_data

def split_data(x_data, y_data):
    # 첫 200개의 데이터를 테스트 데이터로 사용
    x_test = x_data[:200]
    y_test = y_data[:200]
    
    # 나머지 데이터를 학습 데이터로 사용
    x_train = x_data[200:]
    y_train = y_data[200:]
    
    # 표준화 (스케일링)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    return x_train, x_test, y_train, y_test

In [63]:
x_data, y_data = load_csv('fetal_health.csv')
x_train, x_test, y_train, y_test = split_data(x_data, y_data)

In [64]:
learning_rate_RMSProp = 0.01
learning_rate_GradientDescent = 0.1
num_epochs = 40
batch_size = 32
display_step = 10
input_size = x_train.shape[1]
hidden1_size = 64
hidden2_size = 32

In [65]:
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, 3])

In [66]:
def build_autoencoder(x):
  # 인코딩(Encoding)
  Wh_1 = tf.Variable(tf.random_normal([input_size, hidden1_size]))   
  bh_1 = tf.Variable(tf.random_normal([hidden1_size]))
  H1_output = tf.nn.sigmoid(tf.matmul(x, Wh_1) +bh_1)
  Wh_2 = tf.Variable(tf.random_normal([hidden1_size, hidden2_size]))
  bh_2 = tf.Variable(tf.random_normal([hidden2_size]))
  H2_output = tf.nn.sigmoid(tf.matmul(H1_output, Wh_2) +bh_2)
  #디코딩(Decoding)
  Wh_3 = tf.Variable(tf.random_normal([hidden2_size, hidden1_size]))
  bh_3 = tf.Variable(tf.random_normal([hidden1_size]))
  H3_output = tf.nn.sigmoid(tf.matmul(H2_output, Wh_3) +bh_3)
  Wo = tf.Variable(tf.random_normal([hidden1_size, input_size]))
  bo = tf.Variable(tf.random_normal([input_size]))
  X_reconstructed = tf.nn.sigmoid(tf.matmul(H3_output,Wo) + bo)
  
  return X_reconstructed, H2_output

In [67]:
def build_softmax_classifier(x):
  W_softmax = tf.Variable(tf.zeros([hidden2_size, 3]))
  b_softmax = tf.Variable(tf.zeros([3]))
  y_pred = tf.nn.softmax(tf.matmul(x, W_softmax) + b_softmax)

  return y_pred

In [68]:
y_pred, extracted_features = build_autoencoder(x) 
y_true = x
y_pred_softmax = build_softmax_classifier(extracted_features)

In [69]:
# 1. Pre-Training
pretraining_loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))# MSE 손실 함수
pretraining_train_step = tf.train.RMSPropOptimizer(learning_rate_RMSProp).minimize(pretraining_loss)
# 2. Fine-Tuning 
finetuning_loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred_softmax), reduction_indices=[1]))# cross-entropy 손실 함수
finetuning_train_step = tf.train.GradientDescentOptimizer(learning_rate_GradientDescent).minimize(finetuning_loss)

In [70]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  # 전체 배치 개수
  total_batch = len(x_train) // batch_size

  for epoch in range(num_epochs):
    # 모든 배치들에 대해서 최적화를 수행
    for i in range(total_batch):
        start = i * batch_size
        end = start + batch_size
        batch_x, batch_y = x_train[start:end], y_train[start:end]
        _, pretraining_loss_print = sess.run([pretraining_train_step, pretraining_loss], feed_dict={x: batch_x})
    # 지정된 epoch마다 학습결과를 출력
    if epoch % display_step == 0:
      print("Epoch: %d, pretraining_loss: %f" % (epoch, pretraining_loss_print))
  print('')

  for epoch in range(num_epochs):
    # 모든 배치들에 대해서 최적화를 수행
    for i in range(total_batch):
        start = i * batch_size
        end = start + batch_size
        batch_x, batch_y = x_train[start:end], y_train[start:end]
        _, finetuning_loss_print = sess.run([finetuning_train_step, finetuning_loss], feed_dict={x: batch_x,  y: batch_y})
    # 지정된 epoch마다 학습결과를 출력
    if epoch % display_step == 0:
      print("Epoch: %d, finetuning_loss: %f" % (epoch, finetuning_loss_print))

  # 오토인코더+Softmax 분류기 모델의 정확도를 출력
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_pred_softmax,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print("\n정확도(오토인코더+Softmax 분류기): %f" % sess.run(accuracy, feed_dict={x: x_test, y: y_test}))

Epoch: 0, pretraining_loss: 1.032140
Epoch: 10, pretraining_loss: 0.385775
Epoch: 20, pretraining_loss: 0.377588
Epoch: 30, pretraining_loss: 0.373338

Epoch: 0, finetuning_loss: 0.267359
Epoch: 10, finetuning_loss: 0.404640
Epoch: 20, finetuning_loss: 0.443897
Epoch: 30, finetuning_loss: 0.432973

정확도(오토인코더+Softmax 분류기): 0.790000
